In [4]:
# ============================================================
# NEXT-LEVEL PSO INITIALIZATION STUDY
# Research-Grade | Reproducible | GitHub Ready
# ============================================================
https://github.com/IqraSaleem-byte/PSO-Initialization-Techniques_Study

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import qmc, f_oneway

# ============================================================
# 1. CREATE RESULTS DIRECTORY
# ============================================================

os.makedirs("results", exist_ok=True)
print("📁 results/ directory created")

# ============================================================
# 2. BENCHMARK FUNCTIONS
# ============================================================

def sphere(x):
    return np.sum(x**2)

def ackley(x):
    n = len(x)
    return (-20*np.exp(-0.2*np.sqrt(np.sum(x**2)/n))
            - np.exp(np.sum(np.cos(2*np.pi*x))/n)
            + 20 + np.e)

def salomon(x):
    s = np.sum(x**2)
    return 1 - np.cos(2*np.pi*np.sqrt(s)) + 0.1*np.sqrt(s)

# ============================================================
# 3. INITIALIZATION METHODS
# ============================================================

def random_init(n, d, bounds):
    return np.random.uniform(bounds[0], bounds[1], (n, d))

def halton_init(n, d, bounds):
    sampler = qmc.Halton(d=d, scramble=False)
    sample = sampler.random(n)
    return qmc.scale(sample, bounds[0], bounds[1])

def torus_init(n, d, bounds):
    alpha = np.sqrt(2)
    points = np.zeros((n, d))
    for i in range(n):
        for j in range(d):
            points[i, j] = (i * alpha * (j+1)) % 1
    return bounds[0] + points * (bounds[1] - bounds[0])

def lcg_init(n, d, bounds, seed=1):
    m, a, c = 2**31 - 1, 1103515245, 12345
    x = seed
    data = np.zeros((n, d))
    for i in range(n):
        for j in range(d):
            x = (a*x + c) % m
            data[i, j] = x / m
    return bounds[0] + data * (bounds[1] - bounds[0])

def mcg_init(n, d, bounds, seed=1):
    m, a = 2**31 - 1, 16807
    x = seed
    data = np.zeros((n, d))
    for i in range(n):
        for j in range(d):
            x = (a*x) % m
            data[i, j] = x / m
    return bounds[0] + data * (bounds[1] - bounds[0])

# ============================================================
# 4. PSO CORE IMPLEMENTATION
# ============================================================

class PSO:
    def __init__(self, func, bounds, dim, init_method,
                 n_particles=40, max_iter=1000,
                 w_max=0.9, w_min=0.4, c1=2.0, c2=2.0):

        self.func = func
        self.bounds = bounds
        self.dim = dim
        self.n = n_particles
        self.max_iter = max_iter
        self.w_max, self.w_min = w_max, w_min
        self.c1, self.c2 = c1, c2

        self.X = init_method(self.n, self.dim, bounds)
        self.V = np.zeros_like(self.X)

        self.pbest = self.X.copy()
        self.pbest_val = np.array([func(x) for x in self.X])

        idx = np.argmin(self.pbest_val)
        self.gbest = self.pbest[idx].copy()
        self.gbest_val = self.pbest_val[idx]

    def optimize(self):
        history = []

        for t in range(self.max_iter):
            w = self.w_max - (self.w_max - self.w_min) * (t / self.max_iter)

            for i in range(self.n):
                r1, r2 = np.random.rand(), np.random.rand()

                self.V[i] = (w * self.V[i]
                             + self.c1 * r1 * (self.pbest[i] - self.X[i])
                             + self.c2 * r2 * (self.gbest - self.X[i]))

                self.X[i] += self.V[i]
                self.X[i] = np.clip(self.X[i],
                                    self.bounds[0],
                                    self.bounds[1])

                fitness = self.func(self.X[i])
                if fitness < self.pbest_val[i]:
                    self.pbest[i] = self.X[i].copy()
                    self.pbest_val[i] = fitness

            idx = np.argmin(self.pbest_val)
            if self.pbest_val[idx] < self.gbest_val:
                self.gbest = self.pbest[idx].copy()
                self.gbest_val = self.pbest_val[idx]

            history.append(self.gbest_val)

        return self.gbest_val, history

# ============================================================
# 5. EXPERIMENT CONFIGURATION
# ============================================================

functions = {
    "Sphere": (sphere, (-5.12, 5.12)),
    "Ackley": (ackley, (-35, 35)),
    "Salomon": (salomon, (-100, 100))
}

initializers = {
    "BPSO": random_init,
    "HPSO": halton_init,
    "TPSO": torus_init,
    "LCGPSO": lcg_init,
    "MCGPSO": mcg_init
}

DIMENSION = 30
RUNS = 10
ITERATIONS = 1000

# ============================================================
# 6. RUN EXPERIMENTS + SAVE RESULTS
# ============================================================

records = {}
csv_rows = []

for fname, (func, bounds) in functions.items():
    records[fname] = {}

    for method, init in initializers.items():
        best_vals = []
        last_history = None

        for _ in range(RUNS):
            pso = PSO(func, bounds, DIMENSION, init,
                      max_iter=ITERATIONS)
            best, history = pso.optimize()
            best_vals.append(best)
            last_history = history

        mean_val = np.mean(best_vals)
        std_val = np.std(best_vals)

        records[fname][method] = best_vals

        csv_rows.append({
            "Function": fname,
            "Method": method,
            "Mean Best Fitness": mean_val,
            "Std Deviation": std_val
        })

        print(f"{fname} | {method} → Mean: {mean_val:.4e}")

        # Save convergence plot
        plt.figure(figsize=(8,5))
        plt.plot(last_history)
        plt.xlabel("Iterations")
        plt.ylabel("Best Fitness")
        plt.title(f"{method} – {fname}")
        plt.grid(True)
        plt.savefig(f"results/convergence_{fname.lower()}_{method.lower()}.png",
                    dpi=300, bbox_inches="tight")
        plt.close()

# Save CSV
df = pd.DataFrame(csv_rows)
df.to_csv("results/average_results.csv", index=False)

print("\n📊 average_results.csv saved")

# ============================================================
# 7. STATISTICAL ANALYSIS (ANOVA)
# ============================================================

print("\n📐 ANOVA STATISTICAL TEST RESULTS\n")

for fname in records:
    data = list(records[fname].values())
    f_stat, p_val = f_oneway(*data)

    print(f"{fname} → p-value: {p_val:.4e}")
    if p_val < 0.05:
        print("✔ Significant difference found\n")
    else:
        print("✖ No significant difference\n")

print("✅ EXPERIMENT COMPLETED SUCCESSFULLY")


📁 results/ directory created
Sphere | BPSO → Mean: 6.1668e-02
Sphere | HPSO → Mean: 9.4274e-02
Sphere | TPSO → Mean: 1.0572e-01
Sphere | LCGPSO → Mean: 1.1646e-01
Sphere | MCGPSO → Mean: 1.0287e-01
Ackley | BPSO → Mean: 4.1530e+00
Ackley | HPSO → Mean: 1.9982e+01
Ackley | TPSO → Mean: 1.6754e+01
Ackley | LCGPSO → Mean: 7.3652e+00
Ackley | MCGPSO → Mean: 4.3676e+00
Salomon | BPSO → Mean: 1.6399e+00
Salomon | HPSO → Mean: 1.5199e+00
Salomon | TPSO → Mean: 2.4399e+00
Salomon | LCGPSO → Mean: 2.4499e+00
Salomon | MCGPSO → Mean: 1.5999e+00

📊 average_results.csv saved

📐 ANOVA STATISTICAL TEST RESULTS

Sphere → p-value: 2.7132e-01
✖ No significant difference

Ackley → p-value: 5.5352e-12
✔ Significant difference found

Salomon → p-value: 6.1632e-01
✖ No significant difference

✅ EXPERIMENT COMPLETED SUCCESSFULLY


In [5]:
!apt-get install git -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [8]:
!git config --global user.name "IqraSaleem-byte"
!git config --global user.email "iqrasaleem403@gmail.com"

In [9]:
!git clone https://github.com/IqraSaleem-byte/PSO-Initialization-Techniques_Study


Cloning into 'PSO-Initialization-Techniques_Study'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [30]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import modules
import shutil
import os

# Step 3: Define source files and destination folder
# Replace the paths with your actual notebook path in Drive
source_files = [
    "/content/drive/MyDrive/Colab Notebooks/PSO_Initialization_Techniques_Study.ipynb",  # Your notebook
    "/content/drive/MyDrive/Colab Notebooks/results"  # Your results folder
]

dest_folder = "/content/PSO-Initialization-Analysis"  # Local folder to prepare GitHub repo

# Step 4: Create destination folder if it doesn't exist
os.makedirs(dest_folder, exist_ok=True)

# Step 5: Copy files and folders
for f in source_files:
    if os.path.exists(f):
        if os.path.isdir(f):
            shutil.copytree(f, os.path.join(dest_folder, os.path.basename(f)), dirs_exist_ok=True)
            print(f"Directory copied: {f}")
        else:
            shutil.copy(f, dest_folder)
            print(f"File copied: {f}")
    else:
        print(f"Warning: '{f}' not found!")

# Step 6: List all files in destination folder to confirm
print("\nContents of destination folder:")
!ls -R "/content/PSO-Initialization-Analysis"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File copied: /content/drive/MyDrive/Colab Notebooks/PSO_Initialization_Techniques_Study.ipynb

Contents of destination folder:
/content/PSO-Initialization-Analysis:
PSO_Initialization_Techniques_Study.ipynb  results

/content/PSO-Initialization-Analysis/results:
average_results.csv	       convergence_salomon_lcgpso.png
convergence_ackley_bpso.png    convergence_salomon_mcgpso.png
convergence_ackley_hpso.png    convergence_salomon_tpso.png
convergence_ackley_lcgpso.png  convergence_sphere_bpso.png
convergence_ackley_mcgpso.png  convergence_sphere_hpso.png
convergence_ackley_tpso.png    convergence_sphere_lcgpso.png
convergence_salomon_bpso.png   convergence_sphere_mcgpso.png
convergence_salomon_hpso.png   convergence_sphere_tpso.png


In [38]:
# Go to the project folder
%cd /content/PSO-Initialization-Analysis

# Initialize Git
!git init

# Add all files
!git add .

# Commit changes
!git commit -m "Initial commit: PSO notebook + results"

# Rename branch to main (optional, GitHub default)
!git branch -M main

# Add your GitHub repo URL (replace YOUR_USERNAME and REPO_NAME)
!git remote add origin https://github.com/IqraSaleem-byte
# Push to GitHub
!git push -u origin main


/content/PSO-Initialization-Analysis
Reinitialized existing Git repository in /content/PSO-Initialization-Analysis/.git/
On branch main
nothing to commit, working tree clean
error: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address
